# 1.Extraction From Bigquery

In [177]:
from google.colab import auth
auth.authenticate_user()

# Import library BigQuery
from google.cloud import bigquery

# Konfigurasi project ID Google Cloud
project_id = "pipeline-etl-454422"
client = bigquery.Client(project=project_id)

# Test koneksi dengan menampilkan dataset yang tersedia
datasets = list(client.list_datasets())
if datasets:
    print("✅ Koneksi berhasil! Dataset tersedia:")
    for dataset in datasets:
        print(f" - {dataset.dataset_id}")
else:
    print("⚠️ Tidak ada dataset yang ditemukan.")


✅ Koneksi berhasil! Dataset tersedia:
 - ETL


In [178]:
#load dataset
query = "SELECT * FROM `pipeline-etl-454422.ETL.all_years_o3`"

df1 = client.query(query).to_dataframe()
df1.head()

,Date,Country,City,Specie,count,min_ppb,max_ppb,median_ppb
0,2017-02-09,US,Los Angeles,o3,16,0.0,32.0,4.0
1,2020-03-12,US,Los Angeles,o3,16,0.0,19.0,2.0
2,2020-04-28,US,Los Angeles,o3,19,0.0,43.0,10.0
3,2018-03-05,US,Los Angeles,o3,21,0.0,39.0,17.0
4,2020-03-24,US,Los Angeles,o3,23,0.0,31.0,23.0


In [179]:
#load dataset
query = "SELECT * FROM `pipeline-etl-454422.ETL.all_years_pm25`"

df2 = client.query(query).to_dataframe()
df2.head()

,Date,Country,City,Specie,count,min_ugm3,max_ugm3,median_ugm3
0,2020-06-13,US,Manhattan,pm25,9,0.2,3.6,0.9
1,2020-06-14,US,Manhattan,pm25,24,0.2,4.5,2.6
2,2020-06-12,US,Manhattan,pm25,26,0.2,6.0,1.9
3,2017-04-01,US,Los Angeles,pm25,27,0.2,22.0,14.9
4,2020-06-01,US,Manhattan,pm25,32,0.2,5.2,1.9


# 2.Transform

In [180]:
print(df1.describe(),"\n")
print(df2.describe())

            count      min_ppb      max_ppb   median_ppb
count      5030.0  5030.000000  5030.000000  5030.000000
mean   218.072565     5.668191    58.727237    23.969583
std    221.101651    18.221507    95.061138    24.577311
min           3.0     0.000000     2.000000     1.000000
25%          24.0     1.000000    29.000000    15.000000
50%         120.0     2.000000    42.000000    22.000000
75%         387.0     7.000000    57.000000    31.000000
max         888.0   604.000000   604.000000   604.000000 

            count     min_ugm3     max_ugm3  median_ugm3
count      5772.0  5772.000000  5772.000000  5772.000000
mean   216.450277    11.314518    92.522842    32.520444
std    212.483214    20.074735   134.008742    42.090050
min           4.0     0.200000     3.600000     0.900000
25%          71.0     1.200000    21.100000     7.600000
50%         119.0     3.100000    46.000000    14.400000
75%         385.0    13.000000   109.700000    42.800000
max         888.0   328.20000

In [181]:
print(df1.info(), '\n')
print( df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5030 entries, 0 to 5029
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Date        5030 non-null   dbdate 
 1   Country     5030 non-null   object 
 2   City        5030 non-null   object 
 3   Specie      5030 non-null   object 
 4   count       5030 non-null   Int64  
 5   min_ppb     5030 non-null   float64
 6   max_ppb     5030 non-null   float64
 7   median_ppb  5030 non-null   float64
dtypes: Int64(1), dbdate(1), float64(3), object(3)
memory usage: 319.4+ KB
None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5772 entries, 0 to 5771
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         5772 non-null   dbdate 
 1   Country      5772 non-null   object 
 2   City         5772 non-null   object 
 3   Specie       5772 non-null   object 
 4   count        5772 non-null   Int64  
 5

In [182]:
#hapus kolom yang tidak dibutuhkan
df1.drop(df1.columns[[3,5,6,7]], axis=1, inplace=True)
df1.head()

,Date,Country,City,count
0,2017-02-09,US,Los Angeles,16
1,2020-03-12,US,Los Angeles,16
2,2020-04-28,US,Los Angeles,19
3,2018-03-05,US,Los Angeles,21
4,2020-03-24,US,Los Angeles,23


In [183]:
#rename kolom
df1.rename(columns={'count': 'count_o3'}, inplace=True)
df1.head()

,Date,Country,City,count_o3
0,2017-02-09,US,Los Angeles,16
1,2020-03-12,US,Los Angeles,16
2,2020-04-28,US,Los Angeles,19
3,2018-03-05,US,Los Angeles,21
4,2020-03-24,US,Los Angeles,23


In [184]:
#drop juga beberapa kolom di dataset ke 2
df2.drop(df2.columns[[3,5,6,7]], axis=1, inplace=True)
df2.head()

,Date,Country,City,count
0,2020-06-13,US,Manhattan,9
1,2020-06-14,US,Manhattan,24
2,2020-06-12,US,Manhattan,26
3,2017-04-01,US,Los Angeles,27
4,2020-06-01,US,Manhattan,32


In [185]:
#Rename kolom count di df2
df2.rename(columns={'count':'Count_pm25'}, inplace=True)
df2.head()

,Date,Country,City,Count_pm25
0,2020-06-13,US,Manhattan,9
1,2020-06-14,US,Manhattan,24
2,2020-06-12,US,Manhattan,26
3,2017-04-01,US,Los Angeles,27
4,2020-06-01,US,Manhattan,32


In [186]:
#cek duplikat
duplikat1= df1.duplicated().sum()
duplikat2= df2.duplicated().sum()

print('Data 1 : ', duplikat1)
print('Data 2 : ', duplikat2)

Data 1 :  35
Data 2 :  40


In [187]:
#drop data duplikat
df1.drop_duplicates(inplace= True)
df2.drop_duplicates(inplace=True)
print('drop duplikat selesai \n')

drop duplikat selesai 



In [188]:
#Cek ulang duplikat
print(df1.duplicated().sum())
print(df2.duplicated().sum())

0
0


In [189]:
#cek duplikat
misval1= df1.isnull().sum()
misval2= df2.isnull().sum()

print('Data 1 : ', misval1)
print('\n Data 2 : ', misval2)

Data 1 :  Date        0
Country     0
City        0
count_o3    0
dtype: int64

 Data 2 :  Date          0
Country       0
City          0
Count_pm25    0
dtype: int64


In [190]:
#merge 2 data yang sudah di transform
import pandas as pd

new_df = pd.merge(df1, df2,  how='left', left_on=['Date','Country', 'City'], right_on=['Date','Country', 'City'])
new_df.head()

,Date,Country,City,count_o3,Count_pm25
0,2017-02-09,US,Los Angeles,16,50
1,2020-03-12,US,Los Angeles,16,72
2,2020-04-28,US,Los Angeles,19,50
3,2018-03-05,US,Los Angeles,21,66
4,2020-03-24,US,Los Angeles,23,68


# 3.Load to BigQuery

In [191]:
# Schema Table
# CREATE TABLE merge_counts (
#     "Date" DATE,
#     "Country" VARCHAR,
#     "City" VARCHAR,
#     "Count_o3" INT,
#     "Count_pm25" INT
# );

In [160]:
!pip install google-cloud-bigquery pandas pyarro

In [161]:
# Inisialisasi BigQuery Client
project_id = "pipeline-etl-454422"
dataset_id = "ETL"
table_id = "merge_counts"                                                                                                        # Simpan ke BigQuery

#Simpan Ke Bigquery
new_df.to_gbq(f"{dataset_id}.{table_id}", project_id=project_id, if_exists="append")

print("✅ Data berhasil disimpan ke BigQuery!")

<ipython-input-161-ffac352eacd9>:7: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  new_df.to_gbq(f"{dataset_id}.{table_id}", project_id=project_id, if_exists="append")
100%|██████████| 1/1 [00:00<00:00, 4629.47it/s]

✅ Data berhasil disimpan ke BigQuery!
